In [1]:
import pandas as pd
import numpy as np
import PIL
from keras.applications import VGG16,imagenet_utils
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from keras.models import Model
from keras.layers import Dense, Flatten
import numpy as np
import glob
import os

import pickle
import time

Using TensorFlow backend.


In [2]:
bbox = pd.read_table('Data/DeepFashion/list_bbox.txt',sep='\s+')
cat = pd.read_table('Data/DeepFashion/list_category_img.txt',sep='\s+')

In [3]:
joined = bbox.join(cat,lsuffix='_',rsuffix='')

In [4]:
joined.head()

,image_name_,x_1,y_1,x_2,y_2,image_name,category_label
0,img/Sheer_Pleated-Front_Blouse/img_00000001.jpg,72,79,232,273,img/Sheer_Pleated-Front_Blouse/img_00000001.jpg,3
1,img/Sheer_Pleated-Front_Blouse/img_00000002.jpg,67,59,155,161,img/Sheer_Pleated-Front_Blouse/img_00000002.jpg,3
2,img/Sheer_Pleated-Front_Blouse/img_00000003.jpg,65,65,156,200,img/Sheer_Pleated-Front_Blouse/img_00000003.jpg,3
3,img/Sheer_Pleated-Front_Blouse/img_00000004.jpg,51,62,167,182,img/Sheer_Pleated-Front_Blouse/img_00000004.jpg,3
4,img/Sheer_Pleated-Front_Blouse/img_00000005.jpg,46,88,166,262,img/Sheer_Pleated-Front_Blouse/img_00000005.jpg,3


In [5]:
labels = joined['category_label'].value_counts()
top_labels = labels[labels>1000].index.values

In [6]:
subset = pd.DataFrame([], columns=joined.columns)
for l in top_labels:
    has_label = joined[joined['category_label']==l]
    sample = np.random.randint(has_label.shape[0],size=1000)
    subset = subset.append(has_label.iloc[sample])

In [7]:
subset['category_label'] = pd.Categorical(subset['category_label'])
subset['code'] = subset['category_label'].cat.codes

In [8]:
inputShape = (224, 224)
preprocess = imagenet_utils.preprocess_input

def preprocess_DF(bbox):
    image = crop_resize_DF(bbox)
    image = img_to_array(image)
    image = np.expand_dims(image,axis=0)
    
    return preprocess(image)

def crop_resize_DF(bbox):
    img = PIL.Image.open('./Data/DeepFashion/'+bbox['image_name'])
    lx = bbox['x_1']
    ly = bbox['y_1']
    ux = bbox['x_2']
    uy = bbox['y_2']
    
    img = img.crop((lx,ly,ux,uy))
    img = img.resize(inputShape, PIL.Image.ANTIALIAS)
    return img

In [9]:
def generator(df, batch_size):
    while True:
        subset = np.random.randint(df.shape[0],size=batch_size)
        proc_imgs = []
        for i in subset:
            proc_imgs.append(np.squeeze(preprocess_DF(df.iloc[i])))
    
        attr = df.iloc[subset]['code'].values
        yield np.array(proc_imgs), attr

In [10]:
def gen_batch(df,batch_size):
    subset = np.random.randint(df.shape[0],size=batch_size)
    proc_imgs = []
    for i in subset:
        proc_imgs.append(np.squeeze(preprocess_DF(df.iloc[i])))
        print(df.iloc[i]['image_name'],df.iloc[i]['code'])
    
    attr = df.iloc[subset]['code'].values
    return np.array(proc_imgs), attr

In [11]:
attr = subset['category_label'].values

In [12]:
np.bincount(attr)

array([   0,    0, 1000, 1000,    0,    0, 1000,    0,    0,    0, 1000,
       1000,    0,    0,    0,    0, 1000, 1000, 1000, 1000,    0,    0,
          0,    0, 1000,    0, 1000,    0,    0, 1000, 1000,    0, 1000,
       1000, 1000, 1000,    0,    0,    0, 1000,    0, 1000, 1000,    0,
       1000,    0,    0,    0, 1000])

In [13]:
(classes,counts) = np.unique(attr,return_counts=True)

In [14]:
classes

array([ 2,  3,  6, 10, 11, 16, 17, 18, 19, 24, 26, 29, 30, 32, 33, 34, 35,
       39, 41, 42, 44, 48])

In [15]:
class_weights = joined.shape[0]/(counts*len(classes))

In [16]:
class_weights

array([ 13.14645455,  13.14645455,  13.14645455,  13.14645455,
        13.14645455,  13.14645455,  13.14645455,  13.14645455,
        13.14645455,  13.14645455,  13.14645455,  13.14645455,
        13.14645455,  13.14645455,  13.14645455,  13.14645455,
        13.14645455,  13.14645455,  13.14645455,  13.14645455,
        13.14645455,  13.14645455])

In [17]:
from keras import optimizers
from keras import regularizers
from keras.layers.normalization import BatchNormalization
from keras.layers import Dropout
l = 0.01

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output

x = Flatten(name='flatten')(x)
x = BatchNormalization()(x)
x = Dropout(rate=0.2)(x)

x = Dense(4096, activation='relu', name='fc1', kernel_initializer='glorot_normal',
         bias_initializer='glorot_uniform', kernel_regularizer=regularizers.l2(l))(x)
x = BatchNormalization()(x)
x = Dropout(rate=0.2)(x)

predictions = Dense(len(classes), activation='softmax',name='predictions', kernel_initializer='glorot_normal',
         bias_initializer='glorot_uniform', kernel_regularizer=regularizers.l2(l))(x)

new_model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False


new_model.compile(optimizer='Adadelta', loss='sparse_categorical_crossentropy',
                 metrics = ['sparse_categorical_accuracy'])

In [18]:
new_model.load_weights('DF-category-retrain.h5')

In [19]:
from keras import callbacks
tbCB = callbacks.TensorBoard(log_dir="logs/DF_category{}".format(time.time()), histogram_freq=0, write_graph=False, write_images=False)
checkpointCB = callbacks.ModelCheckpoint("checkpoints/DF_category{}".format(time.time()))

h = new_model.fit_generator(generator(subset,128),steps_per_epoch=70,epochs=80,
                            validation_data=generator(subset,128),validation_steps=1,
                            verbose=1,callbacks=[tbCB,checkpointCB])

Epoch 1/80
70/70 [==============================] - 168s - loss: 4.2671 - sparse_categorical_accuracy: 0.6184 - val_loss: 4.1439 - val_sparse_categorical_accuracy: 0.6562
Epoch 2/80
70/70 [==============================] - 168s - loss: 3.4504 - sparse_categorical_accuracy: 0.7382 - val_loss: 2.9864 - val_sparse_categorical_accuracy: 0.7969
Epoch 3/80
70/70 [==============================] - 170s - loss: 2.8829 - sparse_categorical_accuracy: 0.7834 - val_loss: 2.6766 - val_sparse_categorical_accuracy: 0.7812
Epoch 4/80
70/70 [==============================] - 169s - loss: 2.6509 - sparse_categorical_accuracy: 0.7980 - val_loss: 2.4615 - val_sparse_categorical_accuracy: 0.8359
Epoch 5/80
70/70 [==============================] - 168s - loss: 2.5525 - sparse_categorical_accuracy: 0.8090 - val_loss: 2.6361 - val_sparse_categorical_accuracy: 0.8281
Epoch 6/80
70/70 [==============================] - 169s - loss: 2.4985 - sparse_categorical_accuracy: 0.8116 - val_loss: 2.4659 - val_sparse_cat

KeyboardInterrupt: 

In [20]:
new_model.save('DF-Category-trained-7-24.h5')

In [21]:
del new_model

In [ ]:
h = new_model.fit_generator(generator(subset,128),steps_per_epoch=70,epochs=80,
                            validation_data=generator(subset,128),validation_steps=1,
                            verbose=1,callbacks=[tbCB,checkpointCB])

In [ ]:
new_model.save_weights('DF-category-retrain.h5')

In [ ]:
p = new_model.predict_generator(generator(subset,64),steps=1)

In [ ]:
p

In [ ]:
plt.hist()

In [ ]:
np.argmax(p,axis=1)

In [ ]:
counts[17]/np.sum(counts)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.bar(np.arange(len(counts)),counts)

In [ ]:
X, Y = gen_batch(subset,64)

In [ ]:
p = new_model.predict(X)

In [ ]:
np.argmax(p,axis=1)

In [ ]:
Y

In [ ]:
np.sum(np.argmax(p,axis=1)==Y)

In [ ]:
X[0]

In [ ]:
subset